# Aprendizaje basado en instancias

## Procesado de los datos

El conjunto de datos sobre cáncer de mama está incluido en *Scikit-learn*, se obtiene usando la función `load_breast_cancer` incluida en la librería `sklearn.datasets`. Este conjunto de datos contiene 569 ejemplos con 30 características sobre clasificaciones de cáncer de mama, con dos clasificaciones posibles.

In [1]:
from sklearn.datasets import load_breast_cancer

cancer = load_breast_cancer()

Este conjunto de datos es un diccionario con varios campos:
* `data`: Es el conjunto de datos, se trata de un array en el que cada componente es un array con las características de cada instancia.
* `target`: Es el conjunto de valores de clasificación para cada instancia. Es un array del mismo tamaño que `data`, en el que se indica el valor de clasificación de cada instancia, en el mismo orden en que éstas se encuentran en el array `data`.
* `DESCR`: Es una descripción del conjunto de datos.
* `target_names`: Es un array con los nombres de cada valor de clasificación.
* `feature_names`: Es un array con los nombres de cada característica.

Almacenamos los datos en las variables `X_data`, `y_data`, `X_names` e `y_names`.

In [2]:
X_data, y_data, X_names, y_names = \
    cancer.data, cancer.target, cancer.feature_names, cancer.target_names

## Ejemplos aislados (*outliers*)

Un ejemplo aislado (*outlier* en inglés) es un ejemplo de entrenamiento que está rodeado por ejemplos que no pertenecen a su misma clase. La presencia de ejemplos aislados en un conjunto de entrenamiento puede deberse a:

* Un error en los datos
* Una cantidad insuficiente de ejemplos de la clase de los ejemplos aislados
* La existencia de características que no se están teniendo en cuenta
* Clases desbalanceadas 

En el contexto de la clasificación basada en instancias mediante el algoritmo **k**-*NN*, los ejemplos aislados generan ruido que disminuye el rendimiento del clasificador. Dados dos números naturales, $k$ y $r$, con $k>r$, decimos que un ejemplo es ($k$,$r$)-aislado si en sus $k$ vecinos más cercanos hay más de $r$ ejemplos que no pertenecen a su misma clase.

En este ejercicio vamos a identificar ejemplos ($k$,$r$)-aislados en el conjunto de datos y vamos a comparar el rendimiento del clasificador **K**-*NN* con y sin estos ejemplos. Para ello vamos a utilizar la clase `NearestNeighbors` de *Scikit-learn*, incluida en la librería `sklearn.neighbors`.

In [3]:
from sklearn.neighbors import NearestNeighbors

## Contenido del ejercicio

El ejercicio consiste en

* Investigar la clase `NearestNeighbors` de *Scikit-learn* y cómo se usa para obtener los vecinos más cercanos a un ejemplo dado dentro de un conjunto de datos, con respecto a una medida de distancia.
* Definir la función `buscaOutliers` que dados dos números naturales `k` y `r`, devuelve la lista de los índices de los ejemplos del conjunto de entrenamiento sobre el cáncer que son (`k`,`r`)-aislados. Es decir, la lista de los índices de los ejemplos del conjunto de entrenamiento `X_data` para los que en sus `k` vecinos más cercanos hay más de `r` ejemplos con una clasificación distinta a la del propio ejemplo.
* Construir un segundo conjunto de datos `Xo_data` obtenido a partir del inicial eliminando todos los ejemplos (5,3)-aislados.
* Construir dos modelos de decisión (para valores de `p` y `k` coherentes con la búsqueda de ejemplos aislados), uno para cada conjunto de datos considerado `X_data` y `Xo_data`, realizando en ambos casos una separación del $30$% de datos de prueba y $70$% de datos de entrenamiento.
* Comparar el rendimiento de ambos modelos sobre su correspondiente conjunto de prueba.

El **desarrollo tiene que estar razonado**, indicando en cada apartado qué se está haciendo, **demostrando así el conocimiento adquirido en este módulo**. ¿Qué conclusiones puedes sacar de lo aprendido sobre aprendizaje basado en instancias?

### NearestNeighbors
La clase `NearestNeighbors` es un modelo dentro de la familia de 'modelos basado en vecinos cercanos' (k-NN) de tipo no supervisado. Esta clase devuelve el número de vecinos más cercanos en distancia. Este número de vecinos es indicado mediante el parámetro *n_neighbors*. Entre otros de los parámetros que admite se encuentra *algorithm*, que sirve para indicar el algoritmo a utilizar para calcular a los vecinos cercanos. Por defecto tiene el valor *auto* lo que significa que decidirá cual es el mas apropiado en funcion de los valores de ejemplos que se utilizan para el entrenamiento.

### buscaOutliers

A continuación se define la función `buscaOutliers` que va a eliminar aquellas ejemplos que estan rodeados de instancias con diferente clasificacion, es decir, quitar ruido. Dado *k* vecinos y *r* ejemplos, devuelve los índices de los ejemplos del conjunto de entrenamiento en los cuales en sus *k* vecinos mas cercanos hay mas de *r* ejemplos con una clasificación distinta a la del ejemplo. Para ello se va a hacer uso de la clase investigada anteriormente a la cual le indicaremos los *k* vecinos que queremos obtener del conjunto de entrenamiento *X_data*.

Esta clase devuelve un array del tamaño del número de elementos del conjunto de entrenamiento. Cada elemento de este array contiene otro en el que la primera posición es el índice de cada elemento del conjunto de entrenamiento y los siguientes son los índices de sus *k-1* vecinos más cercanos.

Una vez obtenidos estos índices los va a recorrer para obtener la clasificación de cada elemento del conjunto de entrenamiento. Para cada vecino cercano de ese elemento, va a comprobar si la clasificación del vecino es distinta a la del elemento que se esta revisando. En caso afirmativo, lo contabilizará como un ejemplo.

Tras verificar todos los vecinos cercanos del elemento, comprueba si el número total de ejemplos contabilizados es mayor que los *r* ejemplos, si es así lo añadirá a la lista *outliners* que contiene el resultado de esta función con los índices de los ejemplos con una clasificación distinta más cercanos.

In [4]:
import numpy as np
def buscaOutliers(k, r):
    nbrs = NearestNeighbors(n_neighbors=k).fit(X_data)
    distances, indices = nbrs.kneighbors(X_data)
    outliners = []    
    for elements in indices:
        element = elements[0]
        elementClass = y_data[element]
        neighborsClass = 0
        for neighbour in elements[1:]:
            nbClass = y_data[neighbour]
            if nbClass != elementClass:
                neighborsClass += nbClass
        if neighborsClass > r:
            outliners.append(element)
    return outliners


El siguiente paso es eliminar los (5,3)-aislados ejemplos del conjunto de entrenamiento. Utilizando la función `buscaOutliers` obtenemos los índices de los elementos aislados que queremos descartar y mediante el método *delete* de *numpy* los eliminamos tanto del conjunto de entrenamiento *X_data*, obteniendo `Xo_data`; como del conjunto de los valores de clasificación *y_data* obteniendo `yo_data`

In [5]:
outliners = buscaOutliers(5,3)
Xo_data =  np.delete(X_data, outliners, axis=0)
yo_data =  np.delete(y_data, outliners, axis=0)

### Modelos de decisión

Ahora vamos a aplicar el algoritmo k-NN al conjunto de entrenamiento *X_data* y *Xo_data*. En primer lugar dividimos los conjuntos de datos, respectivamente, en un conjunto de prueba y en un conjunto de entrenamiento. Como se indica, para esta división se ha aplicado una separación del 30% para datos de prueba, siendo el 70% restante para entrenamiento.

In [6]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_data,y_data,test_size = 0.30, random_state=4861, stratify=y_data)
Xo_train, Xo_test, yo_train, yo_test = train_test_split(Xo_data,yo_data,test_size = 0.30, random_state=4861, stratify=yo_data)


Una vez tenemos los conjuntos de entrenamiento (\**_train*) y los de pruebas (\**_test*) aplicamos el algoritmo k-NN. El número de vecinos (*n_neighbors*, parámetro *k*) elegido ha sido 5, para mantener la coherencia y poder compararlo con el conjunto de datos *Xo_data*, que no contiene los elementos aislados de los 5 vecinos más cercanos. Para la distancia (parámetro *p*) se ha elegido el valor 1 (distancia de Manhattan) porque las características que se están midiendo (area, textura, simetria..) no tienen nada en común.

Primero lo aplicamos sobre el conjunto de entrenamiento original (*X_data*) y calculamos su rendimiento haciendo uso del método score.

In [7]:
from sklearn.neighbors import KNeighborsClassifier
p = 1
k = 5
knn = KNeighborsClassifier(n_neighbors=k,p=p)
knn.fit(X_data,y_data)
knn.score(X_test,y_test)

0.9649122807017544

A continuación lo aplicamos sobre el conjunto sin los elementos aislados (*Xo_data*) y calculamos el rendimiento.

In [8]:
knno = KNeighborsClassifier(n_neighbors=k,p=p)
knno.fit(Xo_data,yo_data)
knno.score(Xo_test,yo_test)

0.9761904761904762

### Conclusion

Al comparar ambos rendimientos vemos que sobre el conjunto con los ejemplos (5,3)-aislados elimminados (*Xo_data*) ha mejorado, aunque sea levemente, ya que hemos eliminado ruido ayudando de esta forma al modelo. 